# Detecting Deepfakes with DINOv2

## Load & Transform Dataset


In [2]:
!pip install datasets

import os, zipfile, subprocess, glob, shutil
from huggingface_hub import snapshot_download
from datasets import load_dataset

In [3]:
# load dataset from huggingface, due to unusual file storage, it needs to be loaded manually
repo = snapshot_download("pujanpaudel/deepfake_face_classification", repo_type="dataset")
out = os.path.join(repo, "extracted")
os.makedirs(out, exist_ok=True)

def extract(src, dst):
    os.makedirs(dst, exist_ok=True)
    if src.endswith(".zip"):
        with zipfile.ZipFile(src) as z: z.extractall(dst)
    elif src.endswith(".rar"):
        if not shutil.which("unrar"):
            subprocess.run(["apt-get","update"]); subprocess.run(["apt-get","install","-y","unrar-free"])
        subprocess.run(["unrar","x","-o+",src,dst])

def find_root(p):
    imgs = glob.glob(f"{p}/**/*.[jJpP]*[gG]", recursive=True)
    class_dir = os.path.dirname(imgs[0])
    return os.path.dirname(class_dir)

splits = {"train":"train.rar","validation":"val.zip","test":"test.zip"}
datasets = {}

for split, fname in splits.items():
    path = os.path.join(repo, fname)
    if not os.path.exists(path): continue
    dst = os.path.join(out, split)
    extract(path, dst)
    root = find_root(dst)
    datasets[split] = load_dataset("imagefolder", data_dir=root)["train"]
    print(split, datasets[split])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/25696 [00:00<?, ?it/s]

train Dataset({
    features: ['image', 'label'],
    num_rows: 25696
})


Resolving data files:   0%|          | 0/3212 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

validation Dataset({
    features: ['image', 'label'],
    num_rows: 3212
})


Resolving data files:   0%|          | 0/3212 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

test Dataset({
    features: ['image', 'label'],
    num_rows: 3212
})


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
from torchvision import transforms

preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

def preprocess_dataset(example):
    example["pixel_values"] = preprocess(example["image"])
    return example

train_dataset = datasets['train'].map(preprocess_dataset)
val_dataset   = datasets['validation'].map(preprocess_dataset)
#test_dataset  = datasets['test'].map(preprocess_dataset)


Map:   0%|          | 0/3212 [00:00<?, ? examples/s]

## Finetune DINO-v2-small


In [12]:
from transformers import AutoModelForImageClassification
import torch

# Load the pre-trained DINOv2 model
model = AutoModelForImageClassification.from_pretrained("facebook/dinov2-small", num_labels=2)

Some weights of Dinov2ForImageClassification were not initialized from the model checkpoint at facebook/dinov2-small and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=3,
    save_steps=500,
    save_total_limit=2,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_steps=50,
    dataloader_num_workers=12,
    fp16=True,
    optim="adamw_torch",


)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()


Epoch,Training Loss,Validation Loss
1,0.502800,0.296924
2,0.257500,0.150084
3,0.105000,0.090610


TrainOutput(global_step=603, training_loss=0.4090970050339675, metrics={'train_runtime': 1588.8317, 'train_samples_per_second': 48.519, 'train_steps_per_second': 0.38, 'total_flos': 1.5357612203747574e+18, 'train_loss': 0.4090970050339675, 'epoch': 3.0})

In [14]:
from sklearn.metrics import accuracy_score

predictions = trainer.predict(val_dataset)

accuracy = accuracy_score(predictions.label_ids, predictions.predictions.argmax(-1))
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.9704


### Save Model

In [15]:
from google.colab import drive
drive.mount('/content/drive')

import torch
torch.save(model.state_dict(), '/content/drive/MyDrive/deepfake_dino_model_full_128.pth')
print("Model saved to Google Drive.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model saved to Google Drive.


## Ablation: Train head only (vs whole model above)

In [20]:
from transformers import AutoModelForImageClassification
import torch

# Load the pre-trained DINOv2 model
model = AutoModelForImageClassification.from_pretrained("facebook/dinov2-small", num_labels=2)

Some weights of Dinov2ForImageClassification were not initialized from the model checkpoint at facebook/dinov2-small and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
# only fine-tune the last layer
for param in model.parameters():
    param.requires_grad = False
for param in model.classifier.parameters():
    param.requires_grad = True

for name, param in model.named_parameters():
    print(name, param.requires_grad)

dinov2.embeddings.cls_token False
dinov2.embeddings.mask_token False
dinov2.embeddings.position_embeddings False
dinov2.embeddings.patch_embeddings.projection.weight False
dinov2.embeddings.patch_embeddings.projection.bias False
dinov2.encoder.layer.0.norm1.weight False
dinov2.encoder.layer.0.norm1.bias False
dinov2.encoder.layer.0.attention.attention.query.weight False
dinov2.encoder.layer.0.attention.attention.query.bias False
dinov2.encoder.layer.0.attention.attention.key.weight False
dinov2.encoder.layer.0.attention.attention.key.bias False
dinov2.encoder.layer.0.attention.attention.value.weight False
dinov2.encoder.layer.0.attention.attention.value.bias False
dinov2.encoder.layer.0.attention.output.dense.weight False
dinov2.encoder.layer.0.attention.output.dense.bias False
dinov2.encoder.layer.0.layer_scale1.lambda1 False
dinov2.encoder.layer.0.norm2.weight False
dinov2.encoder.layer.0.norm2.bias False
dinov2.encoder.layer.0.mlp.fc1.weight False
dinov2.encoder.layer.0.mlp.fc1.bias

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=3,
    save_steps=500,
    save_total_limit=2,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_steps=50,
    dataloader_num_workers=12,
    fp16=True,
    optim="adamw_torch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,0.472300,0.407770
2,0.400700,0.368274


In [ ]:
from sklearn.metrics import accuracy_score

predictions = trainer.predict(val_dataset)

accuracy = accuracy_score(predictions.label_ids, predictions.predictions.argmax(-1))
print(f"Accuracy: {accuracy:.4f}")

### Save Model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import torch
torch.save(model.state_dict(), '/content/drive/MyDrive/deepfake_dino_model_head_128.pth')
print("Model saved to Google Drive.")
